In [ ]:
!pip install py-entitymatching

In [2]:
# https://hpi.de/naumann/projects/repeatability/datasets/restaurants-dataset.html
# http://www.iesl.cs.umass.edu/datasets.html - CORA dataset
# Importing the libraries
import sys
import py_entitymatching as em
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Display the versions
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 3.7.10 (default, Feb 20 2021, 21:17:23) 
[GCC 7.5.0]
pandas version: 1.1.5
magellan version: 0.4.0


In [4]:
# Load csv files as dataframes and set the key attribute in the dataframe
path_A = "/content/gdrive/My Drive/Colab Notebooks/BTP/music_data/music1.csv"
path_B = "/content/gdrive/My Drive/Colab Notebooks/BTP/music_data/music2.csv"
A = em.read_csv_metadata(path_A, key='Sno')
B = em.read_csv_metadata(path_B, key='Sno')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [5]:
# Display number of tuples in the datasets
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 6907
Number of tuples in B: 21163
Number of tuples in A X B (i.e the cartesian product): 146172841


In [6]:
# Displaying first two entries from the first music dataset
A.head(2)

,Sno,Album_Name,Album_Price,Artist_Name,CopyRight,Customer_Rating,Genre,Price,Released,Song_Name,Time,Group
0,1,Welcome to Cam Country - EP,$4.26,Cam,2015 Sony Music Entertainment,4.72396,"Country,Music,Contemporary Country,Honky Tonk",$0.99,31-Mar-15,Runaway Train,3:01,Country
1,2,Me 4 U,$9.99,Omi,"2015 Ultra Records, LLC under exclusive license to Columbia Records, a Division of Sony Music E...",3.38158,"Pop/Rock,Music,Pop,Dance,R&B/Soul",Album Only,NaN,Track 14,3:41,Pop


In [7]:
# Displaying first two entries from the second music dataset
B.head(2)

,Sno,Album_Name,Artist_Name,Song_Name,Price,Time,Released,Label,Copyright,Genre,Group
0,1,! (Volume 2) [Explicit],Rusko,Saxophone Stomp [Explicit],$1.29,3:20,"September 16, 2014",Decca International,(C) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.,"Dance & Electronic,Dubstep",Dance
1,2,! (Volume 2) [Explicit],Rusko,I Wanna Mingle [feat. Pusher],$1.29,2:36,"September 16, 2014",Decca International,(C) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.,"Dance & Electronic,Dubstep",Dance


**Downsampling**

In [ ]:
# Display the keys of the input tables
print(em.get_key(A), em.get_key(B))
# If the tables are large we can downsample the tables like this
A1, B1 = em.down_sample(A, B, 5000, 1, show_progress=False)
print("Lengths after downsampling-")
print(len(A1), len(B1))

Sno Sno
Lengths after downsampling-
1245 5000


**Blocking**

In [ ]:
# # Create attribute equivalence blocker
# ab = em.AttrEquivalenceBlocker()

# # Block using artist_name attribute
# C = ab.block_tables(A1, B1, "Artist_Name","Artist_Name",
#                     l_output_attrs=["Sno", "Album_Name", "Artist_Name", "CopyRight", "Released", "Song_Name" ,"Time"], 
#                     r_output_attrs=["Sno", "Album_Name", "Artist_Name", "Copyright", "Released", "Song_Name", "Time"]
#                     )


# Overlap based blocker, which blocks on the basis of overlap of some qgram between two strings
ob = em.OverlapBlocker()
C = ob.block_tables(A, B, "Album_Name", "Album_Name", overlap_size=2, 
                    l_output_attrs=["Sno", "Album_Name", "Artist_Name","Genre","Song_Name","CopyRight", "Released", "Price" ,"Time"], 
                    r_output_attrs=["Sno", "Album_Name", "Artist_Name","Genre","Song_Name","Copyright", "Released", "Price" ,"Time"]
                    )


# Rule based blocker, which blocks on the basis of a rule
# rb = em.RuleBasedBlocker()
# block_f = em.get_features_for_blocking(A, B,validate_inferred_attr_types=False)
# # Add more rules
# rule1 = ["Album_Name_Album_Name_lev_sim(ltuple, rtuple) > 0.8"]
# rule2 = ["Song_Name_Song_Name_lev_sim(ltuple, rtuple) > 0.8"]

# rb.add_rule(rule1, block_f)
# rb.add_rule(rule2, block_f)

# C = rb.block_tables(A, B, 
#                     l_output_attrs=["Sno", "Album_Name", "Artist_Name","Genre","Song_Name","CopyRight", "Released", "Price" ,"Time"], 
#                     r_output_attrs=["Sno", "Album_Name", "Artist_Name","Genre","Song_Name","Copyright", "Released", "Price" ,"Time"]
#                     )

/usr/local/lib/python3.7/dist-packages/py_entitymatching/blocker/overlap_blocker.py:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  l_df[l_dummy_overlap_attr] = l_df[l_overlap_attr]
/usr/local/lib/python3.7/dist-packages/py_entitymatching/blocker/overlap_blocker.py:615: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table[overlap_attr] = values
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:19


In [ ]:
# Printing length of candidate set
print(len(C))
print(C.head(2))

3042443
   _id  ltable_Sno  rtable_Sno  ...      rtable_Released rtable_Price rtable_Time
0    0        3996           1  ...   September 16, 2014        $1.29        3:20
1    1        3996           2  ...   September 16, 2014        $1.29        2:36

[2 rows x 19 columns]


In [ ]:
C.head(2)

,_id,ltable_Sno,rtable_Sno,ltable_Album_Name,ltable_Artist_Name,ltable_Genre,ltable_Song_Name,ltable_CopyRight,ltable_Released,ltable_Price,ltable_Time,rtable_Album_Name,rtable_Artist_Name,rtable_Genre,rtable_Song_Name,rtable_Copyright,rtable_Released,rtable_Price,rtable_Time
0,0,3996,1,A Rock By The Sea Christmas :: Volume 2,Brian Fechino,"Holiday,Music,Rock",Rudolph Blue,2011 Hubbub!,8-Nov-11,$0.99,1:57,! (Volume 2) [Explicit],Rusko,"Dance & Electronic,Dubstep",Saxophone Stomp [Explicit],(C) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.,"September 16, 2014",$1.29,3:20
1,1,3996,2,A Rock By The Sea Christmas :: Volume 2,Brian Fechino,"Holiday,Music,Rock",Rudolph Blue,2011 Hubbub!,8-Nov-11,$0.99,1:57,! (Volume 2) [Explicit],Rusko,"Dance & Electronic,Dubstep",I Wanna Mingle [feat. Pusher],(C) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.,"September 16, 2014",$1.29,2:36


In [ ]:
# Overlap based blocker, which blocks on the basis of overlap of some qgram between two strings
C = ob.block_candset(C,"Song_Name", "Song_Name",overlap_size=2)

/usr/local/lib/python3.7/dist-packages/py_entitymatching/blocker/overlap_blocker.py:615: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table[overlap_attr] = values
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:44


In [ ]:
# Printing length of candidate set
print(len(C))
print(C.head(2))

63246
    _id  ltable_Sno  rtable_Sno  ... rtable_Released rtable_Price rtable_Time
9     9         726          28  ...   July 30, 2012        $0.99        8:49
10   10         727          28  ...   July 30, 2012        $0.99        8:49

[2 rows x 19 columns]


In [ ]:
C.head(2)

,_id,ltable_Sno,rtable_Sno,ltable_Album_Name,ltable_Artist_Name,ltable_Genre,ltable_Song_Name,ltable_CopyRight,ltable_Released,ltable_Price,ltable_Time,rtable_Album_Name,rtable_Artist_Name,rtable_Genre,rtable_Song_Name,rtable_Copyright,rtable_Released,rtable_Price,rtable_Time
9,9,726,28,"A Little Less 16 Candles, a Little More ""Touch Me"" - Single",Fall Out Boy,"Alternative,Music,Rock,Punk","A Little Less 16 Candles, a Little More ""Touch Me""",2006 The Island Def Jam Music Group,20-Jun-06,$1.99,6:44,"Notice Me"" 2012 - Little Carlos Retro Mix""","Matthias Heilbronn, Sandee","Electronica,Dance & Electronic,House",Notice Me (Little Carlos Retro Mix),2012 InHouse Records,"July 30, 2012",$0.99,8:49
10,10,727,28,"A Little Less 16 Candles, a Little More ""Touch Me"" - Single",Fall Out Boy,"Alternative,Music,Rock,Punk","A Little Less Sixteen Candles, a Little More ""Touch Me""",2006 The Island Def Jam Music Group,20-Jun-06,$1.29,2:49,"Notice Me"" 2012 - Little Carlos Retro Mix""","Matthias Heilbronn, Sandee","Electronica,Dance & Electronic,House",Notice Me (Little Carlos Retro Mix),2012 InHouse Records,"July 30, 2012",$0.99,8:49


In [ ]:
C = ob.block_candset(C, 'CopyRight', 'Copyright',overlap_size=2)

/usr/local/lib/python3.7/dist-packages/py_entitymatching/blocker/overlap_blocker.py:615: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table[overlap_attr] = values
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


In [ ]:
# Printing length of candidate set
print(len(C))
print(C.head(2))

12829
       _id  ltable_Sno  rtable_Sno  ...  rtable_Released rtable_Price rtable_Time
3173  3173        2584         229  ...   March 20, 2014        $1.29        3:03
3210  3210        2004         230  ...    June 17, 2014        $1.29        3:52

[2 rows x 19 columns]


In [ ]:
C.head(2)

,_id,ltable_Sno,rtable_Sno,ltable_Album_Name,ltable_Artist_Name,ltable_Genre,ltable_Song_Name,ltable_CopyRight,ltable_Released,ltable_Price,ltable_Time,rtable_Album_Name,rtable_Artist_Name,rtable_Genre,rtable_Song_Name,rtable_Copyright,rtable_Released,rtable_Price,rtable_Time
3173,3173,2584,229,#Selfie (Instrumental Mix) - Single,The Chainsmokers,"Dance,Music,Electronic",#Selfie (Instrumental Mix),"2014 Dim Mak Inc. under exclusive license to Republic Records, a Division of UMG Recordings, Inc.",20-Mar-14,$1.29,3:03,#SELFIE (Instrumental Mix),The Chainsmokers,Dance & Electronic,#SELFIE (Instrumental Mix),"(C) 2014 Dim Mak Inc. under exclusive license to Republic Records, a Division of UMG Recordings...","March 20, 2014",$1.29,3:03
3210,3210,2004,230,#SELFIE (The Remixes) - Single,The Chainsmokers,"Electronic,Music,Dance",#SELFIE (Caked Up Remix),"2014 Dim Mak Inc. under exclusive license to Republic Records, a Division of UMG Recordings, In...",17-Jun-14,$1.29,3:15,#Selfie (The Remixes),The Chainsmokers,"Electronica,Dance & Electronic",#Selfie (Will Sparks Remix),"(C) 2014 Dim Mak Inc. under exclusive license to Republic Records, a Division of UMG Recordings...","June 17, 2014",$1.29,3:52


**Feature Generation**

In [ ]:
# Generate features
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

In [ ]:
# List the names of the features generated
feature_table['feature_name']

0                                     Sno_Sno_exm
1                                     Sno_Sno_anm
2                                Sno_Sno_lev_dist
3                                 Sno_Sno_lev_sim
4           Album_Name_Album_Name_jac_qgm_3_qgm_3
5       Album_Name_Album_Name_cos_dlm_dc0_dlm_dc0
6       Album_Name_Album_Name_jac_dlm_dc0_dlm_dc0
7                       Album_Name_Album_Name_mel
8                  Album_Name_Album_Name_lev_dist
9                   Album_Name_Album_Name_lev_sim
10                      Album_Name_Album_Name_nmw
11                       Album_Name_Album_Name_sw
12        Artist_Name_Artist_Name_jac_qgm_3_qgm_3
13    Artist_Name_Artist_Name_cos_dlm_dc0_dlm_dc0
14    Artist_Name_Artist_Name_jac_dlm_dc0_dlm_dc0
15                    Artist_Name_Artist_Name_mel
16               Artist_Name_Artist_Name_lev_dist
17                Artist_Name_Artist_Name_lev_sim
18                    Artist_Name_Artist_Name_nmw
19                     Artist_Name_Artist_Name_sw


In [ ]:
from math import *

def jaccard_sim(x,y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

In [ ]:
# Manually creating features for some attribute

def time_time_lev_sim(ltuple, rtuple):
    p1 = ltuple.Time
    p2 = rtuple.Time
    return em.lev_sim(p1,p2)

def time_time_cos(ltuple, rtuple):
    p1 = ltuple.Time
    p2 = rtuple.Time
    return em.cosine(list(p1),list(p2))

def time_time_jac(ltuple, rtuple):
    p1 = ltuple.Time
    p2 = rtuple.Time
    return jaccard_sim(p1,p2)
    
def released_released_lev_sim(ltuple, rtuple):
    p1 = str(ltuple.Released)
    p2 = str(rtuple.Released)
    if p1=="nan" or p2=="nan":
        return 0
    p1 = p1.split("-") # 31,mar,15
    if len(p1)<3:
        return 0
    d = {"Jan":"January","Feb":"February","Mar":"March","Apr":"April","May":"May","Jun":"June","Jul":"July","Aug":"August","Sep":"September","Oct":"October","Nov":"November","Dec":"December"}
    s1 = " " + d[p1[1]] + " " + p1[0] + ", 20" + p1[2]
    return em.lev_sim(s1,p2)

def released_released_cos(ltuple, rtuple):
    p1 = str(ltuple.Released)
    p2 = str(rtuple.Released)
    if p1=="nan" or p2=="nan":
        return 0
    p1 = p1.split("-") # 31,mar,15
    if len(p1)<3:
        return 0
    d = {"Jan":"January","Feb":"February","Mar":"March","Apr":"April","May":"May","Jun":"June","Jul":"July","Aug":"August","Sep":"September","Oct":"October","Nov":"November","Dec":"December"}
    s1 = " " + d[p1[1]] + " " + p1[0] + ", 20" + p1[2]
    return em.cosine(list(s1),list(p2))

def released_released_jac(ltuple, rtuple):
    p1 = str(ltuple.Released)
    p2 = str(rtuple.Released)
    if p1=="nan" or p2=="nan":
        return 0
    p1 = p1.split("-") # 31,mar,15
    if len(p1)<3:
        return 0
    d = {"Jan":"January","Feb":"February","Mar":"March","Apr":"April","May":"May","Jun":"June","Jul":"July","Aug":"August","Sep":"September","Oct":"October","Nov":"November","Dec":"December"}
    s1 = " " + d[p1[1]] + " " + p1[0] + ", 20" + p1[2]
    return jaccard_sim(s1,p2)
    
def copyright_copyright_lev_sim(ltuple, rtuple):
    p1 = ltuple.CopyRight
    p2 = rtuple.Copyright
    return em.lev_sim(p1,p2)

def copyright_copyright_cos(ltuple, rtuple):
    p1 = ltuple.CopyRight
    p2 = rtuple.Copyright
    return em.cosine(list(p1),list(p2))

def copyright_copyright_jac(ltuple, rtuple):
    p1 = ltuple.CopyRight
    p2 = rtuple.Copyright
    return jaccard_sim(p1,p2)

def price_price_lev_sim(ltuple, rtuple):
    p1 = str(ltuple.Price).strip()
    p2 = str(rtuple.Price).strip()
    return em.lev_sim(p1,p2)

def price_price_cos(ltuple, rtuple):
    p1 = str(ltuple.Price).strip()
    p2 = str(rtuple.Price).strip()
    return em.cosine(list(p1),list(p2))

def price_price_jac(ltuple, rtuple):
    p1 = str(ltuple.Price).strip()
    p2 = str(rtuple.Price).strip()
    return jaccard_sim(p1,p2)

In [ ]:
em.add_blackbox_feature(feature_table, 'time_time_lev_sim', time_time_lev_sim)
em.add_blackbox_feature(feature_table, 'time_time_cos', time_time_cos)
em.add_blackbox_feature(feature_table, 'time_time_jac', time_time_jac)
em.add_blackbox_feature(feature_table, 'released_released_lev_sim', released_released_lev_sim)
em.add_blackbox_feature(feature_table, 'released_released_cos', released_released_cos)
em.add_blackbox_feature(feature_table, 'released_released_jac', released_released_jac)
em.add_blackbox_feature(feature_table, 'copyright_copyright_lev_sim', copyright_copyright_lev_sim)
em.add_blackbox_feature(feature_table, 'copyright_copyright_cos', copyright_copyright_cos)
em.add_blackbox_feature(feature_table, 'copyright_copyright_jac', copyright_copyright_jac)
em.add_blackbox_feature(feature_table, 'price_price_lev_sim', price_price_lev_sim)
em.add_blackbox_feature(feature_table, 'price_price_cos', price_price_cos)
em.add_blackbox_feature(feature_table, 'price_price_jac', price_price_jac)

True

In [ ]:
feature_table['feature_name']

0                                     Sno_Sno_exm
1                                     Sno_Sno_anm
2                                Sno_Sno_lev_dist
3                                 Sno_Sno_lev_sim
4           Album_Name_Album_Name_jac_qgm_3_qgm_3
5       Album_Name_Album_Name_cos_dlm_dc0_dlm_dc0
6       Album_Name_Album_Name_jac_dlm_dc0_dlm_dc0
7                       Album_Name_Album_Name_mel
8                  Album_Name_Album_Name_lev_dist
9                   Album_Name_Album_Name_lev_sim
10                      Album_Name_Album_Name_nmw
11                       Album_Name_Album_Name_sw
12        Artist_Name_Artist_Name_jac_qgm_3_qgm_3
13    Artist_Name_Artist_Name_cos_dlm_dc0_dlm_dc0
14    Artist_Name_Artist_Name_jac_dlm_dc0_dlm_dc0
15                    Artist_Name_Artist_Name_mel
16               Artist_Name_Artist_Name_lev_dist
17                Artist_Name_Artist_Name_lev_sim
18                    Artist_Name_Artist_Name_nmw
19                     Artist_Name_Artist_Name_sw


In [ ]:
# Only take features corresponding to the three sim scores : cosine, Jaccard, lev_sim
feature_table = feature_table.drop([0,1,2,3,6,7,8,10,11,14,15,16,18,19,22,23,24,26,27,30,31,32,34,35],axis=0)

In [ ]:
# ##### code written for restuarant dataset-
# lit = []

# for i in range(44):
#     if(i%8!=5):
#         lit.append(i)
        
# # feature_table = feature_table.drop([28,29,30,36,37,38,39,40,41,42,43],axis=0) 

# feature_table = feature_table.drop(lit,axis=0) 

In [ ]:
# List the names of the features generated
feature_table['feature_name']

4           Album_Name_Album_Name_jac_qgm_3_qgm_3
5       Album_Name_Album_Name_cos_dlm_dc0_dlm_dc0
9                   Album_Name_Album_Name_lev_sim
12        Artist_Name_Artist_Name_jac_qgm_3_qgm_3
13    Artist_Name_Artist_Name_cos_dlm_dc0_dlm_dc0
17                Artist_Name_Artist_Name_lev_sim
20                    Genre_Genre_jac_qgm_3_qgm_3
21                Genre_Genre_cos_dlm_dc0_dlm_dc0
25                            Genre_Genre_lev_sim
28            Song_Name_Song_Name_jac_qgm_3_qgm_3
29        Song_Name_Song_Name_cos_dlm_dc0_dlm_dc0
33                    Song_Name_Song_Name_lev_sim
36                              time_time_lev_sim
37                                  time_time_cos
38                                  time_time_jac
39                      released_released_lev_sim
40                          released_released_cos
41                          released_released_jac
42                    copyright_copyright_lev_sim
43                        copyright_copyright_cos


In [ ]:
# # Load the pre-labeled data
# path_G = './data/labeled_res_data.csv'

# # path_G = em.get_install_path() + os.sep + 'datasets' + os.sep + 'end-to-end' + os.sep + 'restaurants/lbl_restnt_wf1.csv'
# G = em.read_csv_metadata(path_G, key="_id",
#                          ltable=A, rtable=B, 
#                          fk_ltable='ltable.id', fk_rtable='rtable.id')
# print(len(G))

In [ ]:
H = em.extract_feature_vecs(C, feature_table=feature_table)
H

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:34


,_id,ltable_Sno,rtable_Sno,Album_Name_Album_Name_jac_qgm_3_qgm_3,Album_Name_Album_Name_cos_dlm_dc0_dlm_dc0,Album_Name_Album_Name_lev_sim,Artist_Name_Artist_Name_jac_qgm_3_qgm_3,Artist_Name_Artist_Name_cos_dlm_dc0_dlm_dc0,Artist_Name_Artist_Name_lev_sim,Genre_Genre_jac_qgm_3_qgm_3,Genre_Genre_cos_dlm_dc0_dlm_dc0,Genre_Genre_lev_sim,Song_Name_Song_Name_jac_qgm_3_qgm_3,Song_Name_Song_Name_cos_dlm_dc0_dlm_dc0,Song_Name_Song_Name_lev_sim,time_time_lev_sim,time_time_cos,time_time_jac,released_released_lev_sim,released_released_cos,released_released_jac,copyright_copyright_lev_sim,copyright_copyright_cos,copyright_copyright_jac,price_price_lev_sim,price_price_cos,price_price_jac
3173,3173,2584,229,0.413043,0.516398,0.600000,1.0,1.0,1.000000,0.517241,0.0,0.681818,0.600000,0.666667,0.807692,1.00,1.00000,1.000000,1.000000,1.000000,1.000000,0.960784,0.957427,0.916667,1.0,1.0,1.0
3210,3210,2004,230,0.341463,0.516398,0.533333,1.0,1.0,1.000000,0.424242,0.0,0.433333,0.195652,0.250000,0.444444,0.50,0.75000,0.600000,1.000000,1.000000,1.000000,0.977528,0.959715,0.921053,1.0,1.0,1.0
3211,3211,2575,230,0.341463,0.516398,0.533333,1.0,1.0,1.000000,0.424242,0.0,0.433333,0.195652,0.250000,0.444444,0.50,0.75000,0.600000,1.000000,1.000000,1.000000,0.977528,0.959715,0.921053,1.0,1.0,1.0
3212,3212,2576,230,0.341463,0.516398,0.533333,1.0,1.0,1.000000,0.424242,0.0,0.433333,0.195652,0.250000,0.444444,0.50,0.75000,0.600000,1.000000,1.000000,1.000000,0.977528,0.959715,0.921053,1.0,1.0,1.0
3295,3295,2004,231,0.341463,0.516398,0.533333,1.0,1.0,1.000000,0.424242,0.0,0.433333,0.219512,0.288675,0.458333,0.50,0.57735,0.400000,1.000000,1.000000,1.000000,0.977528,0.959715,0.921053,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3015737,3015737,1412,55461,0.102362,0.084515,0.298851,0.0,0.0,0.096774,0.000000,0.0,0.125000,0.110169,0.094491,0.287356,0.50,0.50000,0.333333,0.470588,0.483494,0.315789,0.184000,0.633866,0.416667,1.0,1.0,1.0
3015749,3015749,2589,55461,0.080000,0.077152,0.195402,0.0,0.0,0.161290,0.000000,0.0,0.181818,0.086207,0.084515,0.206897,0.50,0.57735,0.400000,0.411765,0.617213,0.444444,0.408000,0.706188,0.545455,1.0,1.0,1.0
3015750,3015750,3518,55461,0.073171,0.000000,0.206897,0.0,0.0,0.129032,0.000000,0.0,0.000000,0.078947,0.000000,0.275862,0.75,0.75000,0.600000,0.631579,0.741249,0.588235,0.218750,0.789330,0.650000,1.0,1.0,1.0
3015751,3015751,4466,55461,0.068376,0.089087,0.218391,0.0,0.0,0.129032,0.133333,0.0,0.312500,0.084112,0.101015,0.218391,0.50,0.50000,0.333333,0.823529,0.889499,0.800000,0.392000,0.762443,0.605263,1.0,1.0,1.0


In [ ]:
col = H['Genre_Genre_cos_dlm_dc0_dlm_dc0'].unique() # just for testing

In [ ]:
import pickle

dbfile = open('/content/gdrive/My Drive/Colab Notebooks/BTP/feature_matrix', 'wb') 
pickle.dump(H, dbfile) 
dbfile.close()

candidate_set = open('/content/gdrive/My Drive/Colab Notebooks/BTP/C', 'wb') 
pickle.dump(C, candidate_set)
candidate_set.close()

In [ ]:
pp

In [8]:
import pickle

dbfile = open('/content/gdrive/My Drive/Colab Notebooks/BTP/feature_matrix', 'rb')    
cand_file = open('/content/gdrive/My Drive/Colab Notebooks/BTP/C', 'rb')  
S = pickle.load(dbfile)
cand = pickle.load(cand_file)

In [ ]:
# verify values in S
S

In [41]:
cand

,_id,ltable_Sno,rtable_Sno,ltable_Album_Name,ltable_Artist_Name,ltable_Genre,ltable_Song_Name,ltable_CopyRight,ltable_Released,ltable_Price,ltable_Time,rtable_Album_Name,rtable_Artist_Name,rtable_Genre,rtable_Song_Name,rtable_Copyright,rtable_Released,rtable_Price,rtable_Time
3173,3173,2584,229,#Selfie (Instrumental Mix) - Single,The Chainsmokers,"Dance,Music,Electronic",#Selfie (Instrumental Mix),"2014 Dim Mak Inc. under exclusive license to Republic Records, a Division of UMG Recordings, Inc.",20-Mar-14,$1.29,3:03,#SELFIE (Instrumental Mix),The Chainsmokers,Dance & Electronic,#SELFIE (Instrumental Mix),"(C) 2014 Dim Mak Inc. under exclusive license to Republic Records, a Division of UMG Recordings...","March 20, 2014",$1.29,3:03
3210,3210,2004,230,#SELFIE (The Remixes) - Single,The Chainsmokers,"Electronic,Music,Dance",#SELFIE (Caked Up Remix),"2014 Dim Mak Inc. under exclusive license to Republic Records, a Division of UMG Recordings, In...",17-Jun-14,$1.29,3:15,#Selfie (The Remixes),The Chainsmokers,"Electronica,Dance & Electronic",#Selfie (Will Sparks Remix),"(C) 2014 Dim Mak Inc. under exclusive license to Republic Records, a Division of UMG Recordings...","June 17, 2014",$1.29,3:52
3211,3211,2575,230,#SELFIE (The Remixes) - Single,The Chainsmokers,"Electronic,Music,Dance",#SELFIE (Caked Up Remix),"2014 Dim Mak Inc. under exclusive license to Republic Records, a Division of UMG Recordings, In...",17-Jun-14,$1.29,3:15,#Selfie (The Remixes),The Chainsmokers,"Electronica,Dance & Electronic",#Selfie (Will Sparks Remix),"(C) 2014 Dim Mak Inc. under exclusive license to Republic Records, a Division of UMG Recordings...","June 17, 2014",$1.29,3:52
3212,3212,2576,230,#SELFIE (The Remixes) - Single,The Chainsmokers,"Electronic,Music,Dance",#SELFIE (Caked Up Remix),"2014 Dim Mak Inc. under exclusive license to Republic Records, a Division of UMG Recordings, In...",17-Jun-14,$1.29,3:15,#Selfie (The Remixes),The Chainsmokers,"Electronica,Dance & Electronic",#Selfie (Will Sparks Remix),"(C) 2014 Dim Mak Inc. under exclusive license to Republic Records, a Division of UMG Recordings...","June 17, 2014",$1.29,3:52
3295,3295,2004,231,#SELFIE (The Remixes) - Single,The Chainsmokers,"Electronic,Music,Dance",#SELFIE (Caked Up Remix),"2014 Dim Mak Inc. under exclusive license to Republic Records, a Division of UMG Recordings, In...",17-Jun-14,$1.29,3:15,#Selfie (The Remixes),The Chainsmokers,"Electronica,Dance & Electronic",#Selfie (Botnek Remix),"(C) 2014 Dim Mak Inc. under exclusive license to Republic Records, a Division of UMG Recordings...","June 17, 2014",$1.29,3:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3015737,3015737,1412,55461,Beautiful Now (feat. Jon Bellion) [Dirty South Remix] - Single,Zedd,"Dance,Music",Beautiful Now (feat. Jon Bellion) [Dirty South Remix],2015 Interscope Records,16-Jun-15,$1.29,3:18,Young And Beautiful [Lana Del Rey vs. Cedric Gervais] (Cedric Gervais Remix Radio Edit),Cedric Gervais and Lana Del Rey,Alternative Rock,Young And Beautiful [Lana Del Rey vs. Cedric Gervais] (Cedric Gervais Remix Radio Edit),(C) 2013 Lana Del Rey under exclusive licence to Polydor Ltd. (UK). Under exclusive licence to ...,"October 29, 2013",$1.29,3:47
3015749,3015749,2589,55461,Let You Go (Radio Edit) [feat. Great Good Fine Ok] - Single,The Chainsmokers,"Dance,Music,Electronic",Let You Go (Radio Edit) [feat. Great Good Fine Ok],"2015 Dim Mak Inc. under exclusive license to Republic Records, a Division of UMG Recordings, Inc.",3-Mar-15,$1.29,3:38,Young And Beautiful [Lana Del Rey vs. Cedric Gervais] (Cedric Gervais Remix Radio Edit),Cedric Gervais and Lana Del Rey,Alternative Rock,Young And Beautiful [Lana Del Rey vs. Cedric Gervais] (Cedric Gervais Remix Radio Edit),(C) 2013 Lana Del Rey under exclusive licence to Polydor Ltd. (UK). Under exclusive licence to ...,"October 29, 2013",$1.29,3:47
3015750,3015750,3518,55461,Can't Believe It (feat. Pitbull) [Radio Edit] - Single,Flo Rida,"Hip-Hop/

In [9]:
import numpy as np
import torch

S = torch.tensor(S.values,dtype=torch.double)
S = S[:,3:]
# S = S.to_numpy(dtype=np.longdouble)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initial weights would be 1/(num_attributes*num_sim_scores) due to add-to-1 constraint
w_init  = (1 / S.shape[1] ) ** 2
# W = np.full((S.shape[0], S.shape[1]), w_init,dtype=np.longdouble)
W = torch.DoubleTensor(S.shape[0],S.shape[1]).fill_(w_init)

In [10]:
# ith row of RLS matrix will return RLS value of the ith record pair 
# It should be a diagonal matrix, as all other values would correspond to RLS for pairs which are not together.
def get_RLS(W,S):
    rls = torch.matmul(S,torch.transpose(W, 0, 1))
    return torch.diag(rls)


# this will return a 1D vector of size n , having value which is to be multiplied to 2*w*S 
# value at ith index would be multiplied to ith pair's derivative
def get_precomputed_values(W,S,phi,tau):
    numerator = torch.exp(phi*get_RLS(W,S)) - torch.exp(phi* (tau-get_RLS(W,S)))
    denominator = torch.exp(phi*get_RLS(W,S)) + torch.exp(phi* (tau-get_RLS(W,S)))
    return torch.divide(numerator,denominator)

# def derivative_loss_function(S,W,j,precomputed_value):
#     partial_deriv = 0
#     for i in range(S.shape[0]):
#         num = 2 * S[j,:].dot(W[j,:]) * ( torch.exp(phi * S[i,:].dot(W[i,:])) - torch.exp(phi * (tau - S[i,:].dot(W[i,:]) )) )
#         denom = torch.exp(phi * S[i,:].dot(W[i,:])) + torch.exp(phi * (tau - S[i,:].dot(W[i,:])))
#         partial_deriv += num/denom
#     return partial_deriv


def derivative_loss_function(W,S,j,precomputed_values):
    return 2 * S[j,:] * W[j,:] * precomputed_values[j]

def algorithm_gradient_descent(W,S,learning_rate,iterations,phi,tau):
    for i in range(iterations):
        W_new = torch.zeros(W.shape) 
        sum_W_new = 0
        precomputed_values = get_precomputed_values(W,S,phi,tau)
        for j in range(W.shape[0]):
            W_new[j,:] = W[j,:] - learning_rate * derivative_loss_function(W,S,j,precomputed_values)
            W_new[j,:] = W_new[j,:] / torch.sum(W_new[j,:])
        W = W_new.double()
        print("Iteration " + str(i+1) + " completed")
    return W

In [11]:
weights = algorithm_gradient_descent(W,S,0.001,200,-50,1)

Iteration 1 completed
Iteration 2 completed
Iteration 3 completed
Iteration 4 completed
Iteration 5 completed
Iteration 6 completed
Iteration 7 completed
Iteration 8 completed
Iteration 9 completed
Iteration 10 completed
Iteration 11 completed
Iteration 12 completed
Iteration 13 completed
Iteration 14 completed
Iteration 15 completed
Iteration 16 completed
Iteration 17 completed
Iteration 18 completed
Iteration 19 completed
Iteration 20 completed
Iteration 21 completed
Iteration 22 completed
Iteration 23 completed
Iteration 24 completed
Iteration 25 completed
Iteration 26 completed
Iteration 27 completed
Iteration 28 completed
Iteration 29 completed
Iteration 30 completed
Iteration 31 completed
Iteration 32 completed
Iteration 33 completed
Iteration 34 completed
Iteration 35 completed
Iteration 36 completed
Iteration 37 completed
Iteration 38 completed
Iteration 39 completed
Iteration 40 completed
Iteration 41 completed
Iteration 42 completed
Iteration 43 completed
Iteration 44 complet

In [12]:
get_RLS(weights,S)

tensor([0.8426, 0.7402, 0.7402,  ..., 0.3421, 0.3676, 0.3570],
       dtype=torch.float64)

In [ ]:
weights.shape

In [14]:
weight_file = open('/content/gdrive/My Drive/Colab Notebooks/BTP/weights', 'wb') 
pickle.dump(weights, weight_file) 
weight_file.close()

In [15]:
weight_file = open('/content/gdrive/My Drive/Colab Notebooks/BTP/weights', 'rb') 
weights = pickle.load(weight_file)

In [37]:
######### Skew metric adopted from https://arxiv.org/pdf/1905.01989.pdf (refer this paper for the metric)

import numpy as np

rls_matrix = get_RLS(weights,S)
ltable_ids = torch.tensor(cand['ltable_Sno'].values)
rtable_ids = torch.tensor(cand['rtable_Sno'].values)

pairs_rls = torch.cat((ltable_ids.reshape(-1,1),rtable_ids.reshape(-1,1),rls_matrix.reshape(-1,1)),dim=1)
pairs_rls = pairs_rls.numpy()
sorted_rls = pairs_rls[np.argsort(-pairs_rls[:,2])] # sorts in descending order
# sorted_array = an_array[numpy.argsort(an_array[:, 1])] (sort by col2)

# Assuming equal representation for all groups
# Assuming same as original representation for all groups

In [38]:
sorted_rls

array([[1.24400000e+03, 3.66890000e+04, 9.42145261e-01],
       [1.24300000e+03, 3.66890000e+04, 9.42145261e-01],
       [1.19100000e+03, 3.66890000e+04, 9.42145261e-01],
       ...,
       [6.17700000e+03, 3.20200000e+03, 1.70989269e-01],
       [4.51000000e+02, 4.91010000e+04, 1.63897386e-01],
       [6.17700000e+03, 3.18900000e+03, 1.51266634e-01]])